In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  # Arrange GPU devices starting from 0
os.environ["CUDA_VISIBLE_DEVICES"]= "0"  # Set the GPU 2 to use
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
tf.config.list_physical_devices('GPU')
# tf.debugging.set_log_device_placement(False)

2024-07-20 13:31:55.594760: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-20 13:31:55.859562: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-20 13:31:55.859592: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-20 13:31:55.860798: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-20 13:31:55.971536: I tensorflow/core/platform/cpu_feature_g

Num GPUs Available:  1


2024-07-20 13:31:58.935298: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-20 13:31:59.054811: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-20 13:31:59.055035: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [36]:
!pip3 install numpy==1.19.5
!pip3 install keras==2.4.3
!pip3 install tensorflow==2.14

  Using cached numpy-1.19.5.zip (7.3 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for numpy (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [926 lines of output]
      setup.py:67: RuntimeWarning: NumPy 1.19.5 may not yet support Python 3.10.
        warnings.warn(
      Running from numpy source directory.
      Cythonizing sources
      numpy/random/_bounded_integers.pxd.in has not changed
      numpy/random/_bounded_integers.pyx.in has not changed
      numpy/random/_sfc64.pyx has not changed
      numpy/random/_generator.pyx has not changed
      numpy/random/_mt19937.pyx has not changed
      numpy/random/_common.pyx has not changed
      numpy/random/mtrand.pyx has not changed
      numpy/random/bit_generator.pyx has not changed
      Processing numpy/random/_bounded_integers.pyx
      numpy/random/_pcg64.pyx h

In [38]:
import matplotlib.pyplot as plt
import numpy as np

In [116]:
n_class = 3
boxes=[3,5,5,5,3,3]
scale = np.linspace(0.2, 0.9, 6)

In [128]:
from tensorflow.keras.layers import Layer
import keras.backend as K

class AnchorBoxes(Layer):
    def __init__(self, w, h, scale, ratios, **kwargs):
        self.w = w
        self.h = h
        self.scale = scale
        self.ratios = ratios
        super(AnchorBoxes, self).__init__(**kwargs)
    def call(self, input):
        size = min(self.w,self.h)
        box_sizes = []
        for r in self.ratios:
            box_sizes.append((self.scale * self.w * np.sqrt(r), self.scale * self.h / np.sqrt(r)))

        box_sizes = np.array(box_sizes)
        
        batch_size, feature_height, feature_width, channel = input.shape

        step_height = self.h / feature_height
        step_width = self.w / feature_width

        cy = np.linspace(0.5 * step_height, (0.5 + feature_height - 1) * step_height, feature_height)
        cx = np.linspace(0.5 * step_width, (0.5 + feature_width - 1) * step_width, feature_width)
        cx_grid, cy_grid = np.meshgrid(cx, cy)
        cx_grid = np.expand_dims(cx_grid, -1)
        cy_grid = np.expand_dims(cy_grid, -1)

        boxes_xywh = np.zeros(shape=(feature_height, feature_width, len(self.ratios), 4))
        boxes_xywh[..., 0] = np.tile(cx_grid, (1, 1, len(self.ratios)))
        boxes_xywh[..., 1] = np.tile(cy_grid, (1, 1, len(self.ratios)))
        boxes_xywh[..., 2] = box_sizes[:, 0]
        boxes_xywh[..., 3] = box_sizes[:, 1]

        boxes_xywh[..., [0, 2]] /= self.w
        boxes_xywh[..., [1, 3]] /= self.h

        boxes_xywh = np.expand_dims(boxes_xywh, axis=0)
        boxes_xywh = K.tile(K.constant(boxes_xywh, dtype='float32'), (tf.shape(input)[0],1,1,1,1))
        return boxes_xywh

In [129]:
from keras.layers import Conv2D,Input, MaxPool2D, Reshape, Concatenate
from keras.regularizers import l2
from keras.models import Model

def ssd():
    input = Input(shape=(300,300,3))
    c1_1 = Conv2D(64,(3,3),padding='same',activation='relu', kernel_initializer='he_normal', kernel_regularizer=l2(0.0005), name='c1_1')(input)
    c1_2 = Conv2D(64,(3,3),padding='same',activation='relu', kernel_initializer='he_normal', kernel_regularizer=l2(0.0005), name='c1_2')(c1_1)
    p1 = MaxPool2D((2,2), (2,2),padding='same',name='p1')(c1_2)

    c2_1 = Conv2D(128,(3,3),padding='same',activation='relu', kernel_initializer='he_normal', kernel_regularizer=l2(0.0005), name='c2_1')(p1)
    c2_2 = Conv2D(128,(3,3),padding='same',activation='relu', kernel_initializer='he_normal', kernel_regularizer=l2(0.0005), name='c2_2')(c2_1)
    p2 = MaxPool2D((2,2), (2,2),padding='same',name='p2')(c2_2)

    c3_1 = Conv2D(256,(3,3),padding='same',activation='relu', kernel_initializer='he_normal', kernel_regularizer=l2(0.0005), name='c3_1')(p2)
    c3_2 = Conv2D(256,(3,3),padding='same',activation='relu', kernel_initializer='he_normal', kernel_regularizer=l2(0.0005), name='c3_2')(c3_1)
    c3_3 = Conv2D(256,(3,3),padding='same',activation='relu', kernel_initializer='he_normal', kernel_regularizer=l2(0.0005), name='c3_3')(c3_2)
    p3 = MaxPool2D((2,2), (2,2),padding='same',name='p3')(c3_3)

    c4_1 = Conv2D(512,(3,3),padding='same',activation='relu', kernel_initializer='he_normal', kernel_regularizer=l2(0.0005), name='c4_1')(p3)
    c4_2 = Conv2D(512,(3,3),padding='same',activation='relu', kernel_initializer='he_normal', kernel_regularizer=l2(0.0005), name='c4_2')(c4_1)
    c4_3 = Conv2D(512,(3,3),padding='same',activation='relu', kernel_initializer='he_normal', kernel_regularizer=l2(0.0005), name='c4_3')(c4_2)
    p4 = MaxPool2D((2,2), (2,2),padding='same',name='p4')(c4_3)

    c5_1 = Conv2D(512,(3,3),padding='same',activation='relu', kernel_initializer='he_normal', kernel_regularizer=l2(0.0005), name='c5_1')(p4)
    c5_2 = Conv2D(512,(3,3),padding='same',activation='relu', kernel_initializer='he_normal', kernel_regularizer=l2(0.0005), name='c5_2')(c5_1)
    c5_3 = Conv2D(512,(3,3),padding='same',activation='relu', kernel_initializer='he_normal', kernel_regularizer=l2(0.0005), name='c5_3')(c5_2)

    c6 = Conv2D(1024,(3,3),padding='same',activation='relu', kernel_initializer='he_normal', kernel_regularizer=l2(0.0005), name='c6')(c5_3)
    c7 = Conv2D(1024,(1,1),padding='same',activation='relu', kernel_initializer='he_normal', kernel_regularizer=l2(0.0005), name='c7')(c6)

    c8_1 = Conv2D(256,(1,1),padding='same',activation='relu', kernel_initializer='he_normal', kernel_regularizer=l2(0.0005), name='c8_1')(c7)
    c8_2 = Conv2D(512,(3,3), strides=(2,2),padding='same',activation='relu', kernel_initializer='he_normal', kernel_regularizer=l2(0.0005), name='c8_2')(c8_1)

    c9_1 = Conv2D(128,(1,1),padding='same',activation='relu', kernel_initializer='he_normal', kernel_regularizer=l2(0.0005), name='c9_1')(c8_2)
    c9_2 = Conv2D(256,(3,3), strides=(2,2),padding='same',activation='relu', kernel_initializer='he_normal', kernel_regularizer=l2(0.0005), name='c9_2')(c9_1)

    c10_1 = Conv2D(128,(1,1),padding='same',activation='relu', kernel_initializer='he_normal', kernel_regularizer=l2(0.0005), name='c10_1')(c9_2)
    c10_2 = Conv2D(256,(3,3),padding='valid',activation='relu', kernel_initializer='he_normal', kernel_regularizer=l2(0.0005), name='c10_2')(c10_1)

    c11_1 = Conv2D(128,(1,1),padding='same',activation='relu', kernel_initializer='he_normal', kernel_regularizer=l2(0.0005), name='c11_1')(c10_2)
    c11_2 = Conv2D(256,(3,3),padding='valid',activation='relu', kernel_initializer='he_normal', kernel_regularizer=l2(0.0005), name='c11_2')(c11_1)

    con1 = Conv2D(boxes[0] * n_class,(3,3),padding='same',activation='relu', kernel_initializer='he_normal', kernel_regularizer=l2(0.0005), name='con1')(c4_3)
    con2 = Conv2D(boxes[1] * n_class,(3,3),padding='same',activation='relu', kernel_initializer='he_normal', kernel_regularizer=l2(0.0005), name='con2')(c7)
    con3 = Conv2D(boxes[2] * n_class,(3,3),padding='same',activation='relu', kernel_initializer='he_normal', kernel_regularizer=l2(0.0005), name='con3')(c8_2)
    con4 = Conv2D(boxes[3] * n_class,(3,3),padding='same',activation='relu', kernel_initializer='he_normal', kernel_regularizer=l2(0.0005), name='con4')(c9_2)
    con5 = Conv2D(boxes[4] * n_class,(3,3),padding='same',activation='relu', kernel_initializer='he_normal', kernel_regularizer=l2(0.0005), name='con5')(c10_2)
    con6 = Conv2D(boxes[5] * n_class,(3,3),padding='same',activation='relu', kernel_initializer='he_normal', kernel_regularizer=l2(0.0005), name='con6')(c11_2)

    loc1 = Conv2D(boxes[0] * 4,(3,3),padding='same',activation='relu', kernel_initializer='he_normal', kernel_regularizer=l2(0.0005), name='loc1')(c4_3)
    loc2 = Conv2D(boxes[1] * 4,(3,3),padding='same',activation='relu', kernel_initializer='he_normal', kernel_regularizer=l2(0.0005), name='loc2')(c7)
    loc3 = Conv2D(boxes[2] * 4,(3,3),padding='same',activation='relu', kernel_initializer='he_normal', kernel_regularizer=l2(0.0005), name='loc3')(c8_2)
    loc4 = Conv2D(boxes[3] * 4,(3,3),padding='same',activation='relu', kernel_initializer='he_normal', kernel_regularizer=l2(0.0005), name='loc4')(c9_2)
    loc5 = Conv2D(boxes[4] * 4,(3,3),padding='same',activation='relu', kernel_initializer='he_normal', kernel_regularizer=l2(0.0005), name='loc5')(c10_2)
    loc6 = Conv2D(boxes[5] * 4,(3,3),padding='same',activation='relu', kernel_initializer='he_normal', kernel_regularizer=l2(0.0005), name='loc6')(c11_2)

    anchor1 = AnchorBoxes(300, 300, scale[0], [1, 2, 0.5], name='anchor1')(loc1)
    anchor2 = AnchorBoxes(300, 300, scale[1], [1, 2, 0.5, 3, 1.0 / 3.0], name='anchor2')(loc2)
    anchor3 = AnchorBoxes(300, 300, scale[2], [1, 2, 0.5, 3, 1.0 / 3.0], name='anchor3')(loc3)
    anchor4 = AnchorBoxes(300, 300, scale[3], [1, 2, 0.5, 3, 1.0 / 3.0], name='anchor4')(loc4)
    anchor5 = AnchorBoxes(300, 300, scale[4], [1, 2, 0.5], name='anchor5')(loc5)
    anchor6 = AnchorBoxes(300, 300, scale[5], [1, 2, 0.5], name='anchor6')(loc6)

    con1 = Reshape((-1, n_class), name='con1_re')(con1)
    con2 = Reshape((-1, n_class), name='con2_re')(con2)
    con3 = Reshape((-1, n_class), name='con3_re')(con3)
    con4 = Reshape((-1, n_class), name='con4_re')(con4)
    con5 = Reshape((-1, n_class), name='con5_re')(con5)
    con6 = Reshape((-1, n_class), name='con6_re')(con6)

    loc1 = Reshape((-1, 4), name='loc1_re')(loc1)
    loc2 = Reshape((-1, 4), name='loc2_re')(loc2)
    loc3 = Reshape((-1, 4), name='loc3_re')(loc3)
    loc4 = Reshape((-1, 4), name='loc4_re')(loc4)
    loc5 = Reshape((-1, 4), name='loc5_re')(loc5)
    loc6 = Reshape((-1, 4), name='loc6_re')(loc6)

    anchor1 = Reshape((-1, 4), name='anchor1_re')(anchor1)
    anchor2 = Reshape((-1, 4), name='anchor2_re')(anchor2)
    anchor3 = Reshape((-1, 4), name='anchor3_re')(anchor3)
    anchor4 = Reshape((-1, 4), name='anchor4_re')(anchor4)
    anchor5 = Reshape((-1, 4), name='anchor5_re')(anchor5)
    anchor6 = Reshape((-1, 4), name='anchor6_re')(anchor6)

    con = Concatenate(axis=1, name='con_con')([con1,con2,con3,con4,con5,con6])
    loc = Concatenate(axis=1, name='loc_con')([loc1,loc2,loc3,loc4,loc5,loc6])
    anchor = Concatenate(axis=1, name='anchor_con')([anchor1,anchor2,anchor3,anchor4,anchor5,anchor6])

    predice = Concatenate(axis=2, name='output')([con, loc, anchor])

    return Model(input, predice)

In [130]:
model = ssd()
print(model.summary())

Model: "model_13"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_39 (InputLayer)       [(None, 300, 300, 3)]        0         []                            
                                                                                                  
 c1_1 (Conv2D)               (None, 300, 300, 64)         1792      ['input_39[0][0]']            
                                                                                                  
 c1_2 (Conv2D)               (None, 300, 300, 64)         36928     ['c1_1[0][0]']                
                                                                                                  
 p1 (MaxPooling2D)           (None, 150, 150, 64)         0         ['c1_2[0][0]']                
                                                                                           